In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
print(selenium.__version__)

4.21.0


In [11]:
# Read data from excel
# Read the Excel file
file_path = 'Book2.xlsx'  # Replace with your file path
sheet_name = 'scraping'  # Replace with your sheet name
column_name_city = 'city_scrape'  # Replace with your column name
column_name_country = 'country_scrape'  # Replace with your column name

# Load the Excel file
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert the column to a list
city_list = df[column_name_city].tolist()
country_list = df[column_name_country].tolist()
population = df['Population'].tolist()

# final list
city_names = []
city_names_bis = []
country_names = []
population_list = []
for idx,city in enumerate(city_list):
    city_names.append(city_list[idx])
    city_names_bis.append(str(city_list[idx])+'-'+str(country_list[idx]))
    country_names.append(country_list[idx])
    population_list.append(population[idx])


FileNotFoundError: [Errno 2] No such file or directory: 'Book2.xlsx'

In [106]:
df.head()
len(df['city'].unique())


710

In [107]:
df.shape

(710, 17)

In [108]:
print(population_list)

[175110, 78703, 112276, 418495, 3415811, 257359, 465138, 101009, 288461, 22615, 2106734, 682605, 1276000, 1075800, 1295714, 2360241, 381488, 607665, 197451, 232930, 5031195, 2141834, 261896, 292630, 130585, 101403, 210165, 155021, 2223236, 65127, 2300500, 75000, 635000, 79550, 1400689, 110000, 342461, 9228000, 358717, 1995471, 536079, 1235192, 201816, 265086, 195278, 110939, 57169, 17222, 679012, 114551, 24307, 1867673, 185042, 580000, 113169, 76390, 419957, 107000, 235884, 2817068, 1773733, 537213, 1437237, 1324781, 2063547, 1332570, 1488920, 2418005, 1897551, 50000, 210813, 119596, 346893, 143325, 115241, 1547779, 142746, 348594, 1143202, 2129371, 147866, 2440462, 656480, 1306784, 138479, 1151635, 439819, 9974, 522888, 438366, 338503, 717961, 3675219, 335949, 1068821, 733156, 266141, 568322, 5647656, 2426160, 30623, 758515, 27704, 889231, 49782, 49360, 24241, 332875, 4766296, 10646000, 9498863, 5606374, 7964000, 10368000, 5621593, 1326588, 893040, 2471474, 22738, 806378, 2529403, 339

In [109]:
# Initialize WebDriver

# Set up Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up the WebDriver with the Chrome options
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()  # Assuming you're using Chrome

# Base URL
url_base = 'https://www.numbeo.com/quality-of-life/in/'

# Initialize an empty DataFrame
df_capital_all = pd.DataFrame(columns=[
    'index',
    'country',
    'city',
    'population',
    'climate_index',
    'cost_of_living_index',
    'health_care_index',
    'pollution_index',
    'property_price_to_income_ratio',
    'purchasing_power_index',
    'safety_index',
    'traffic_commute_time_index',
    'quality_of_life_index'
])

In [110]:
# Fetch data using Selenium

for idx,city in enumerate(city_names):
    
    try:

        url_city = url_base + city_names[idx]
        driver.get(url_city)

        # Find Table element
        tbl_quality = driver.find_element(By.XPATH, '/html/body/div[2]/table')  # if there's no ID, always a good idea to copy xpath

        
        # Parse Table element into pandas DataFrame
        tbl_quality_html = tbl_quality.get_attribute('outerHTML')
        soup = BeautifulSoup(tbl_quality_html, 'html.parser')

        # Find all table rows in the parsed HTML
        tbl_quality_rows = soup.find_all('tr')

        # Initialize data list
        data_cost = []
        for row in tbl_quality_rows:
                row_data = []
                cells = row.find_all(['td', 'th'])
                for cell in cells:
                    row_data.append(cell.get_text().strip())
                data_cost.append(row_data)

        # Extract header and data rows separately
        header = ['kpi_name','kpi_value','kpi_hml']

        # Create DataFrame
        df_capital = pd.DataFrame(data_cost, columns=header)

        # Clean data
        df_capital['Index'] = idx
        df_capital['Country'] = country_names[idx]
        df_capital['City'] = city_names[idx]
        df_capital['Population'] =population_list[idx] 
        df_capital.drop(columns=['kpi_hml'], inplace=True)
        df_capital['kpi_value'] = pd.to_numeric(df_capital['kpi_value'], errors='coerce')
        # df_capital = df_capital[~df_capital['kpi_name'].isna()].copy()
        df_capital.drop(8, inplace=True)


        # Transpose df
        df_capital_t = df_capital.pivot_table(index=['Index','Country','City','Population'], columns='kpi_name', values='kpi_value', dropna=False)
        df_capital_t.reset_index(inplace=True)
        df_capital_t.columns = df_capital_all.columns

        # Append data to complete DF
        df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)

    except:

        try:

            url_city = url_base + city_names_bis[idx]
            driver.get(url_city)

            # Find Table element
            tbl_quality = driver.find_element(By.XPATH, '/html/body/div[2]/table')  # if there's no ID, always a good idea to copy xpath

            
            # Parse Table element into pandas DataFrame
            tbl_quality_html = tbl_quality.get_attribute('outerHTML')
            soup = BeautifulSoup(tbl_quality_html, 'html.parser')

            # Find all table rows in the parsed HTML
            tbl_quality_rows = soup.find_all('tr')

            # Initialize data list
            data_cost = []
            for row in tbl_quality_rows:
                    row_data = []
                    cells = row.find_all(['td', 'th'])
                    for cell in cells:
                        row_data.append(cell.get_text().strip())
                    data_cost.append(row_data)

            # Extract header and data rows separately
            header = ['kpi_name','kpi_value','kpi_hml']

            # Create DataFrame
            df_capital = pd.DataFrame(data_cost, columns=header)

            # Clean data
            df_capital['Index'] = idx
            df_capital['Country'] = country_names[idx]
            df_capital['City'] = city_names_bis[idx]
            df_capital['Population'] =population_list[idx]
            df_capital.drop(columns=['kpi_hml'], inplace=True)
            df_capital['kpi_value'] = pd.to_numeric(df_capital['kpi_value'], errors='coerce')
            # df_capital = df_capital[~df_capital['kpi_name'].isna()].copy()
            df_capital.drop(8, inplace=True)


            # Transpose df
            df_capital_t = df_capital.pivot_table(index=['Index','Country','City'], columns='kpi_name', values='kpi_value', dropna=False)
            df_capital_t.reset_index(inplace=True)
            df_capital_t.columns = df_capital_all.columns

            # Append data to complete DF
            df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)

        except:
             
             # Append NANs
            emtpy_dict = {
            'index':[idx]
            ,'city':[city_names[idx]]
            ,'country':[country_names[idx]]
            ,'population':[population_list[idx]]
            ,'climate_index':[pd.NA]
            ,'cost_of_living_index':[pd.NA]
            ,'health_care_index':[pd.NA]
            ,'pollution_index':[pd.NA]
            ,'property_price_to_income_ratio':[pd.NA]
            ,'purchasing_power_index':[pd.NA]
            ,'safety_index':[pd.NA]
            ,'traffic_commute_time_index':[pd.NA]
            ,'quality_of_life_index':[pd.NA]
            }
            df_capital_t = pd.DataFrame.from_dict(emtpy_dict)
            df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)


C:\Users\paude\AppData\Local\Temp\ipykernel_17424\3440943671.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)


In [94]:
df_capital_all.head(10)

,index,country,city,population,climate_index,cost_of_living_index,health_care_index,pollution_index,property_price_to_income_ratio,purchasing_power_index,safety_index,traffic_commute_time_index,quality_of_life_index


In [78]:
# Merge Net Migration from original df

df_capital_all.reset_index(drop=True, inplace=True)
# df_capital_all.drop_duplicates(subset='city', inplace=True)
df_capital_all = df_capital_all.merge(df['city_net_migration'], left_index=True, right_index=True, how='left')

# Drop entries with NAN in all indexes
index_cols = [
    'climate_index'	
    ,'cost_of_living_index'
    ,'health_care_index'
    ,'pollution_index'
    ,'property_price_to_income_ratio'
    ,'purchasing_power_index'
    ,'safety_index'
    ,'traffic_commute_time_index'
    ,'quality_of_life_index'
    ]

df_capital_all = df_capital_all.dropna(subset=index_cols, how='all')
df_capital_all.head(10)

,index,country,city,population,climate_index,cost_of_living_index,health_care_index,pollution_index,property_price_to_income_ratio,purchasing_power_index,safety_index,traffic_commute_time_index,quality_of_life_index,city_net_migration_x,city_net_migration_y
0,0,Afghanistan,Kabul,4273156.0,NaN,NaN,26.16,89.71,9.61,NaN,22.09,56.17,NaN,-111660.585408,-111660.585408
1,1,Albania,Durres-Albania,175110.0,86.28,NaN,11.11,77.24,9.33,NaN,71.60,30.00,NaN,1391.384603,1391.384603
2,2,Albania,Elbasan-Albania,78703.0,86.28,NaN,30.56,0.00,7.35,NaN,50.55,80.00,NaN,-368.258300,-368.258300
3,3,Albania,Shkoder,112276.0,88.56,NaN,55.00,19.14,10.07,NaN,30.10,14.00,NaN,-2011.951538,-2011.951538
4,4,Albania,Tirana,418495.0,86.43,45.40,47.75,86.97,15.56,40.07,58.96,38.97,95.85,-3622.062041,-3622.062041
5,5,Algeria,Algiers,3415811.0,94.82,30.21,57.15,67.75,26.85,28.78,48.42,51.20,89.55,-60588.512417,-60588.512417
6,6,Algeria,Annaba-Algeria,257359.0,97.28,NaN,76.85,95.69,12.93,NaN,36.56,15.00,NaN,-3444.202627,-3444.202627
7,7,Algeria,Constantine-Algeria,465138.0,84.71,NaN,33.33,54.74,24.33,NaN,55.95,58.67,NaN,-917.403254,-917.403254
8,8,Algeria,Oran-Algeria,101009.0,96.35,28.35,55.32,63.65,17.26,33.43,43.76,105.00,74.47,-2289.192245,-2289.192245
9,9,Algeria,Setif-Algeria,288461.0,80.17,NaN,59.26,72.41,NaN,NaN,44.71,13.50,NaN,-2586.042016,-2586.042016


In [83]:
len(df_capital_all['city'].unique())

829

In [82]:
df_capital_all.shape

(847, 15)

In [49]:
df_capital_all.drop_duplicates(subset='city', inplace=True)

In [84]:
df[df['city_scrape']=='Buenos-Aires'].head()

,index,country,city,Lat,Lng,iso2,iso3,Population,MIGRATION AS A % OF POPULATION/ country,total_net_migration_2022 / country,net_mig_per_thousand_people_2023 / country,Data Year 2,Data Year 3,Population Growth Rate (PGR) Year 2 to Year 3 (%),city_net_migration,country_scrape,city_scrape
12,17.833333,Argentina,Buenos-Aires,-34.6033,-58.3817,AR,ARG,16710000.0,5.048542,3886.0,-0.1,2010.0,2020.0,0.382962,-379464.178016,Argentina,Buenos-Aires
899,NaN,Argentina,Buenos Aires,-34.6033,-58.3817,AR,ARG,16710000.0,5.048542,3886.0,-0.1,2010.0,2020.0,0.382962,-1677.399298,Argentina,Buenos-Aires


In [51]:
df_capital_all.shape

(890, 13)

In [52]:
df_capital_all.to_csv('test_out_final_3.csv')

In [44]:
# Define the file path
file_path = r'C:\Users\paude\Documents\00. Backup 2023\EXTERIOR\MASTER\MA IAAC\03. Cursada\01. Modulo 03\S01 DE\Class Project\DE_Team\Scraped_csvs\data_migration.csv'

# Export DataFrame to a CSV file
df_capital_all.to_csv(file_path, index=False)

print("DataFrame exported successfully to:", file_path)

DataFrame exported successfully to: C:\Users\paude\Documents\00. Backup 2023\EXTERIOR\MASTER\MA IAAC\03. Cursada\01. Modulo 03\S01 DE\Class Project\DE_Team\Scraped_csvs\data_migration.csv
